In [0]:
%pip install xlrd>=2.0.1
%pip install openpyxl
%restart_python

In [0]:
# ---- 1) Imports ----
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Se o spark não existir (em alguns ambientes locais), cria:
try:
    spark
except NameError:
    spark = SparkSession.builder.getOrCreate()

# ---- 2) Leitura do Excel ----
# Suba o arquivo para /FileStore/tables/ no Databricks ou ajuste este caminho:
file_path = "/Workspace/Users/ahmad.jmazloum@gmail.com/Data__1_.xlsx"

df = pd.read_excel(file_path)

# (Opcional) Converter para Spark para visualização no Databricks
try:
    df_spark = spark.createDataFrame(df)
    display(df_spark)  # display() é nativo do Databricks
except Exception as e:
    print("display() indisponível fora do Databricks. Prosseguindo em pandas.", e)

# ---- 3) Inspeção inicial ----
print("Formato (linhas, colunas):", df.shape)
print("\nTipos de dados:")
print(df.dtypes)
print("\nPrimeiras linhas:")
print(df.head())

print("\nValores ausentes por coluna:")
print(df.isnull().sum())

print("\nResumo estatístico:")
print(df.describe())

if "Injury" not in df.columns:
    raise ValueError("A coluna 'Injury' não foi encontrada. Verifique o nome exato no Excel.")

print("\nDistribuição de Injury (como está no arquivo):")
print(df["Injury"].value_counts(dropna=False))

# ---- 4) Limpeza e Normalização ----
# Ajuste os nomes conforme seu arquivo. Estes são os nomes esperados:
expected_cols = [
    "Rush Hour",
    "Alcohol Involved",
    "Work Zone",
    "Align",
    "Weekday",
    "Accident at Intersection",
    "Accident at Roadway",
    "Speed Limit",
    "Number of Vehicle Involved",
    "Weather",
    "Injury"
]

# Conferência de colunas faltantes (apenas alerta)
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    print("\n[AVISO] As colunas abaixo não foram encontradas no dataset e podem precisar de ajuste de nome:")
    print(missing)

df_clean = df.copy()

# Mapear colunas de Yes/No -> 1/0 (se vierem como string)
yes_no_cols = []
for c in ["Alcohol Involved", "Work Zone", "Weekday", "Weather"]:
    if c in df_clean.columns:
        if df_clean[c].dtype == "object":
            yes_no_cols.append(c)

for c in yes_no_cols:
    df_clean[c] = (
        df_clean[c]
        .astype(str)
        .str.strip()
        .str.lower()
        .map({
            "yes": 1, "y": 1, "sim": 1, "s": 1, "true": 1, "1": 1,
            "no": 0, "n": 0, "nao": 0, "não": 0, "false": 0, "0": 0
        })
    )

# Converter Injury de {1,2} para {1,0} onde 1=ferido, 2=não ferido
if df_clean["Injury"].dtype != "int64" and df_clean["Injury"].dtype != "float64":
    df_clean["Injury"] = pd.to_numeric(df_clean["Injury"], errors="coerce")

df_clean["Injury"] = df_clean["Injury"].replace({2: 0})

# Garantir numérico nas colunas principais
cols_to_numeric = [c for c in expected_cols if c in df_clean.columns]
for c in cols_to_numeric:
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")

print("\nValores ausentes após conversões:")
print(df_clean.isnull().sum())

# Estratégia simples: remover linhas com NaN em qualquer coluna usada
df_clean = df_clean.dropna(subset=[c for c in expected_cols if c in df_clean.columns]).reset_index(drop=True)

print("\nAmostra após limpeza:")
print(df_clean.head())

# ---- 5) EDA (Análise Exploratória) ----
# Distribuição do alvo
try:
    sns.countplot(x="Injury", data=df_clean)
    plt.title("Distribuição de Injury (0=sem ferido, 1=ferido)")
    plt.show()
except Exception as e:
    print("Plot não exibido (ambiente sem backend gráfico).", e)

# Correlação
try:
    plt.figure(figsize=(10, 6))
    corr = df_clean.corr(numeric_only=True)
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="viridis")
    plt.title("Correlação entre variáveis")
    plt.show()
except Exception as e:
    print("Heatmap não exibido (ambiente sem backend gráfico).", e)

# Exemplo: taxa média de feridos por 'Alcohol Involved' (se existir)
if "Alcohol Involved" in df_clean.columns:
    print("\nProbabilidade média de ferido por 'Alcohol Involved':")
    print(
        df_clean.groupby("Alcohol Involved")["Injury"]
        .mean()
        .rename("Probabilidade de ferido")
    )

    try:
        sns.barplot(
            x="Alcohol Involved",
            y="Injury",
            data=df_clean,
            estimator=np.mean
        )
        plt.title("Taxa média de feridos vs Álcool envolvido")
        plt.ylabel("Probabilidade de ferido")
        plt.show()
    except Exception as e:
        print("Barplot não exibido.", e)

# ---- 6) Definir X e y ----
feature_cols = [c for c in expected_cols if c != "Injury" and c in df_clean.columns]
if not feature_cols:
    raise ValueError("Nenhuma coluna preditora válida encontrada após a limpeza.")

X = df_clean[feature_cols]
y = df_clean["Injury"]

# ---- 7) Train/Test split ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# ---- 8) Treinamento: Regressão Logística e Random Forest ----
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# ---- 9) Avaliação ----
def avaliar_modelo(y_true, y_pred, nome_modelo):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n=== {nome_modelo} ===")
    print(f"Acurácia : {acc:.3f}")
    print(f"Precisão : {prec:.3f}")
    print(f"Recall   : {rec:.3f}")
    print(f"F1-score : {f1:.3f}")
    print("\nRelatório de Classificação:")
    print(classification_report(y_true, y_pred, zero_division=0))

  # Matriz de confusão
    try:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(4, 3))
        sns.heatmap(cm, annot=True, fmt="d", cbar=False)
        plt.title(f"Matriz de Confusão - {nome_modelo}")
        plt.xlabel("Predito")
        plt.ylabel("Real")
        plt.show()
    except Exception as e:
        print("Não foi possível exibir a matriz de confusão.", e)